# This document goes through the data cleaning of your data set. 


### __Objectives__:
- Remove the np.nan.
- Transform all the data into numerical values. 
- Remove the outliers to have less distorsion. 
- Merge and create new variables using the One Hot Encoding. 

*** 

### __to do__: 
1. ~~Create repository~~
2. ~~Cleaning the code and divide between data cleaning, modelisation~~
3. ~~Create modalization based on the preprocessing~~
4. Create de README.MD file 
5. ~~Clear and finishing the presentation support~~ 
6. Try to modelize based on correlation between independant variables

***

## A. Importing libraries 

In [1]:
import pandas as pd
import numpy as np 
import os 
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

## B. Importing the csv 

In [2]:
path_data_set = os.path.join(os.path.abspath(''), 'data_cleaned.csv')
df = pd.read_csv(path_data_set, sep=";")

In [3]:
print(f"The orignal shape of our file is: {df.shape}. ")

The orignal shape of our file is: (17347, 35). 


In [4]:
df.isna().sum()

ItemID                                 0
Locality                               0
Type of property                       0
Subtype of property                    0
Price                                  0
Number of bedrooms                   453
Livable surface                     1928
Kitchen equipment                      0
Number of bathrooms                    0
Number of toilets                      0
Floor of appartment                13109
Number of floors                    9831
Balcony                                0
State of the property                  0
Build Year                          8212
Surface bedroom 1                   1756
Furnished                              0
Surface of living-room              1723
Cellar                                 0
Surface kitchen                     1825
Entry phone                            0
Elevator                               0
Number of showers                      0
Orientation of the front facade    12146
Number of facade

In [5]:
df.dtypes

ItemID                               int64
Locality                            object
Type of property                    object
Subtype of property                 object
Price                                int64
Number of bedrooms                 float64
Livable surface                    float64
Kitchen equipment                   object
Number of bathrooms                  int64
Number of toilets                    int64
Floor of appartment                float64
Number of floors                   float64
Balcony                              int64
State of the property               object
Build Year                         float64
Surface bedroom 1                   object
Furnished                            int64
Surface of living-room              object
Cellar                               int64
Surface kitchen                     object
Entry phone                          int64
Elevator                             int64
Number of showers                    int64
Orientation

## C. Cleaning variables

### 1. Creating a zip variable

We are creating a new variable called zip.

This variable is taking only the postcode as a value. 

In [6]:
df["zip"] = df["Locality"].str.extract(r"^(^\d+)")
df["zip"] = df["zip"].astype(int)

### 2. Province variable 

For each province in Belgium (10), we are creating a new variable using the One Hot Encoding technique. 

In [7]:
df["Brussels"] = np.where(df["zip"].between(1000,1299),1, 0)
df["Walloon Brabant"] = np.where(df["zip"].between(1300,1499),1, 0)
df["Flemish Brabant"] = np.where(df["zip"].between(1500,1999),1, 0)
df["Flemish Brabant"] = np.where(df["zip"].between(3000,3499),1, df["Flemish Brabant"])
df["Antwerp"] = np.where(df["zip"].between(2000,2999),1, 0)
df["Limburg"] = np.where(df["zip"].between(3500,3999),1, 0)
df["Liege"] = np.where(df["zip"].between(4000,4999),1, 0)
df["Namur"] = np.where(df["zip"].between(5000,5999),1, 0)
df["Hainaut"] = np.where(df["zip"].between(6000,6599),1, 0)
df["Hainaut"] = np.where(df["zip"].between(7000,7999),1, df["Hainaut"])
df["Luxembourg"] = np.where(df["zip"].between(6600,6999),1, 0)
df["West Flanders"] = np.where(df["zip"].between(8000,8999),1, 0)
df["East Flanders"] = np.where(df["zip"].between(9000,9999),1, 0)

We have just created 11 news variables. 1 zip and 10 provinces.

In [8]:
print(f"The new shape of our dataframe is: {df.shape}")

The new shape of our dataframe is: (17347, 47)


### 3. Cleaning Number of bedrooms

We are removing high unexplicable numbers and replacing the np.nan in order to have a int variable.

We are removing 453 items. They have no values for Number of bedrooms.
We are removing 4 items with absude high values.

In [9]:
df = df.drop(df[df['Number of bedrooms'].isna()].index)
df = df.drop(df[df['Number of bedrooms'] > 20].index)

In [10]:
df["Number of bedrooms"] = df["Number of bedrooms"].astype(int)

### 4. Cleaning Number of bathrooms

Removing when higher than '6'. 

We are removing 23 items.

In [11]:
df = df.drop(df[df['Number of bathrooms'] > 6].index)

In [12]:
df['Number of bathrooms'] = df["Number of bathrooms"].astype(int)

### 5. Cleaning Livable surface 

- We are removing all the np.nan because an item cannot have 0m² livable surface. 
- Removed the outliers (more than 2000m²) because they are not representative of the normal real estate marekt.

We are removing more than 1200 items. 

In [13]:
df = df.drop(df[df['Livable surface'].isna()].index)
df = df.drop(df[df['Livable surface'] > 2000].index)

In [14]:
df["Livable surface"] = df["Livable surface"].astype(int)

### 6. Cleaning Kitchen equipment 

- This is an ordinal variables. We can turn it into a numerical ordinal variable.
- We decided to put super and fully at the same level because we are not sure what the difference is.


- Not equipped = 0
- Partially equipped = 1
- Super / fully equipped = 2 

In [15]:
df["Kitchen equipment"] = np.where(df["Kitchen equipment"] == "Not equipped",0, df["Kitchen equipment"])
df["Kitchen equipment"] = np.where(df["Kitchen equipment"] == "Partially equipped",1, df["Kitchen equipment"])
df["Kitchen equipment"] = np.where(df["Kitchen equipment"] == "Super equipped",2, df["Kitchen equipment"])
df["Kitchen equipment"] = np.where(df["Kitchen equipment"] == "Fully equipped",2, df["Kitchen equipment"])

In [16]:
df["Kitchen equipment"] = df["Kitchen equipment"].astype(int)

### 7. Cleaning the Surface of living-room 

- We remove the item when the surface of living-room is higher than livable surface because it doesn't make any sens.
- We also decided to remove all item with a surface of living-room higher than 500.

In [17]:
df["Surface of living-room"] = df["Surface of living-room"].str.replace(',','.').astype(float)
df["Surface of living-room"] = df["Surface of living-room"].apply(np.ceil)
df["Surface of living-room"] = df["Surface of living-room"].astype(int)

In [18]:
df = df.drop(df[df["Surface of living-room"] > df["Livable surface"]].index)
df = df.drop(df[df['Surface of living-room'] > 500].index)

### 8. Cleaning Surface garden 

- We are comparing this variable with Garden.
- Surface garden is False if Garden is also nul. 
- If Garden is True and Surface garden is False, we are changing the value with the value of the 1st quartile.
- We are also removing the value of outliers (> 4000m²)

In [19]:
df = df.drop(df[df['Surface garden'] > 4000].index)
df['Surface garden'] = np.where((df["Garden"] == 0), 0, df["Surface garden"])

In [20]:
df[(df["Surface garden"] > 0) & (df["Surface garden"].notna())].describe()["Surface garden"]

count    2567.000000
mean      380.529801
std       585.085113
min         1.000000
25%        65.000000
50%       150.000000
75%       414.000000
max      4000.000000
Name: Surface garden, dtype: float64

In [21]:
df['Surface garden'] = np.where((df["Surface garden"].isna()) & (df["Garden"] == 1),65, df['Surface garden'])
df["Surface garden"] = df["Surface garden"].astype(int)

### 9. Cleaning Surface terrace

- We are comparing this variable with Terrace.
- Surface terrace is False if Garden is also False.
- If Terrace is True and Surface Terrace is False, we are changing the value with the value of the 1st quartile.


In [22]:
df["Surface terrace"] = np.where((df["Terrace"] == 0),0, df["Surface terrace"])

In [23]:
df[(df["Surface terrace"] > 0) & (df["Surface terrace"].notna())].describe()["Surface terrace"]

count    6382.000000
mean       22.530868
std        46.095996
min         1.000000
25%         8.000000
50%        15.000000
75%        25.000000
max      2769.000000
Name: Surface terrace, dtype: float64

In [24]:
df["Surface terrace"] = np.where(df['Surface terrace'].isna(), 8, df["Surface terrace"])
df["Surface terrace"] = df["Surface terrace"].astype(int)

### 10. Cleaning Number of showers

- Removing the outliers 

In [25]:
df = df.drop(df[df["Number of showers"] > 6].index)

### 11. Cleaning Number of toilets 

- Removing outliers 

In [26]:
df = df.drop(df[df['Number of toilets'] > 6].index)

### 12. Cleaning State of property 

- This is an ordinal variables. We can turn it into a numerical ordinal variable.
- We decided to put excellent and fully renovated at the same level because we are not sure what the difference is.


- To be renovated = 0
- Normal = 1
- Excellent / fully renovated = 2 
- New = 3

In [27]:
df["State of the property"] = np.where(df["State of the property"] == "To be renovated", 0, df["State of the property"])
df["State of the property"] = np.where(df["State of the property"] == "Normal", 1, df["State of the property"])
df["State of the property"] = np.where(df["State of the property"] == "Excellent", 2, df["State of the property"])
df["State of the property"] = np.where(df["State of the property"] == "Fully renovated", 2, df["State of the property"])
df["State of the property"] = np.where(df["State of the property"] == "New", 3, df["State of the property"])
df["State of the property"] = df["State of the property"].astype(int)

### 13. Cleaning Surface kitchen 

- We remove item when Surface kitchen higher than Livable surface. 
- We remove outliers (top 10 percentile). 

In [28]:
df["Surface kitchen"] = df["Surface kitchen"].str.replace(',','.').astype(float)
df["Surface kitchen"].apply(np.ceil)
df["Surface kitchen"] = df["Surface kitchen"].astype(int)
df = df.drop(df[df['Surface kitchen'] > df['Livable surface']].index)

In [29]:
df["Surface kitchen"].describe(percentiles=np.arange(0,1,0.1))

count    14993.000000
mean        15.615887
std         11.953638
min          0.000000
0%           0.000000
10%          7.000000
20%          8.000000
30%         10.000000
40%         11.000000
50%         12.000000
60%         15.000000
70%         17.000000
80%         20.000000
90%         27.000000
max        176.000000
Name: Surface kitchen, dtype: float64

In [30]:
df = df.drop(df[df["Surface kitchen"]>27].index)

### 14. Cleaning Facade orientation

- We are creating a new variable called Orientation South when the orientation is South.

In [31]:
df["Orientation of the front facade"].value_counts()

South         1041
South-west     764
South-east     539
West           511
East           450
North-west     342
North-east     319
North          307
Name: Orientation of the front facade, dtype: int64

In [32]:
df["Orientation South"] = np.where(df["Orientation of the front facade"] == "South", 1, 0)
df["Orientation South"] = np.where(df["Orientation of the front facade"] == "South-west", 1, df["Orientation South"])
df["Orientation South"] = np.where(df["Orientation of the front facade"] == "South-east", 1, df["Orientation South"])

In [33]:
df["Orientation South"].value_counts()

0    11285
1     2344
Name: Orientation South, dtype: int64

### 15. Cleaning Build Year

- Change by hand 3 missed values.
- We are removing this column because a lot of missed value and not good solution to replace it. 

In [34]:
df['Build Year'] = np.where(df['Build Year'] == 22018, 2018, df['Build Year'])
df['Build Year'] = np.where(df['Build Year'] == 20121, 2012, df['Build Year'])
df['Build Year'] = np.where(df['Build Year'] == 19654, 1965, df['Build Year'])

In [35]:
df = df.drop(columns=["Build Year"])

### 16. Cleaning Number of facades

- Remove when the number if higher than 4 because it makes no sens. 

In [36]:
df = df.drop(df[df["Number of facades"] > 4].index)

### 17. Cleaning Type of property 

- We are making a new variable using the One Hot Encoding method. 
- House = 1 
- Appartment = 0

In [37]:
df['Type property'] = np.where(df['Type of property'] == "house", 1,0)
df['Type property'].value_counts()

0    7376
1    6253
Name: Type property, dtype: int64

### 18. Cleaning Surface bedroom 1

- Remove when higher than Livable surface
- Remove outliers when higher than 80m² 
- Remove one item by hand

In [38]:
df["Surface bedroom 1"] = df["Surface bedroom 1"].str.replace(',','.').astype(float)
df['Surface bedroom 1'] = df["Surface bedroom 1"].apply(np.ceil)
df['Surface bedroom 1'] = df['Surface bedroom 1'].astype(int)

In [39]:
df = df.drop(df[df["Surface bedroom 1"]>df["Livable surface"]].index)
df = df.drop(1034)
df = df.drop(df[df["Surface bedroom 1"]> 80].index)

## D. Removing useless and duplicates variables

### 1. Removing useless variables 

In [40]:
df = df.drop(columns=["ItemID", 'Locality', "Subtype of property", "Floor of appartment", "Orientation of the front facade"])

### 2. Removing non-exploitable variables 

In [41]:
df = df.drop(columns=["Elevator",
                     "Surface bedroom 2",
                     "Surface bedroom 3",
                     "Number of floors",
                     "Entry phone"])

### 3. Removing duplicate variables

In [42]:
df = df.drop(columns=["zip",
                "Type of property",
                "Garden",
                "Terrace"])

## E. Description after cleaning

In [43]:
df.shape

(13618, 34)

In [44]:
df.dtypes

Price                     int64
Number of bedrooms        int32
Livable surface           int32
Kitchen equipment         int32
Number of bathrooms       int32
Number of toilets         int64
Balcony                   int64
State of the property     int32
Surface bedroom 1         int32
Furnished                 int64
Surface of living-room    int32
Cellar                    int64
Surface kitchen           int32
Number of showers         int64
Number of facades         int64
Surface terrace           int32
Security door             int64
Access for disabled       int64
Sewer Connection          int64
Surface garden            int32
Garage                    int64
Brussels                  int32
Walloon Brabant           int32
Flemish Brabant           int32
Antwerp                   int32
Limburg                   int32
Liege                     int32
Namur                     int32
Hainaut                   int32
Luxembourg                int32
West Flanders             int32
East Fla

## F. Saving the file into a CSV file

In [45]:
df.to_csv('data_cleaned_reg.csv')